# WAM Industry Domain Industry Data Exploration

## Imports

In [65]:
import pandas as pd
from IPython.display import display, HTML, Markdown
import matplotlib.pyplot as plt

## Load Industries

In [9]:
df_tree = (
    pd.read_feather('./data/gd-mvcr8intel-stage-gd-studio/gd-studio-slogans/df_tree_export_categories_parents.fth')   
    .assign(full_path=lambda df: (df['parents_path'] + '/' + df['name']).str.strip('/'))
    .loc[lambda df: df['visible']]
)
df_tree

,tree,group,name,parent,parents_path,id,photos,visible,full_path
0,Active Living,sports,Active Living,,,active,"[101641, 101642, 101643, 101660, 103620, 10746...",True,Active Living
1,Archery,sports,Archery,active,Active Living,archery,"[101647, 101648, 21289, 21290, 21291, 21292, 2...",True,Active Living/Archery
2,Archery and Shooting,sports,Archery and Shooting,active,Active Living,archeryshooting,"[101647, 101648, 21289, 21290, 21291, 21292, 2...",True,Active Living/Archery and Shooting
3,Gun Club,guns,Gun Club,archeryshooting,Active Living/Archery and Shooting,gunclub,"[103264, 103265, 103269, 103271, 83196, 83197]",True,Active Living/Archery and Shooting/Gun Club
4,Bowling,hobby,Bowling,active,Active Living,bowling,"[4046, 4047, 4048, 4049, 4050, 4055, 4057, 405...",True,Active Living/Bowling
...,...,...,...,...,...,...,...,...,...
1676,Waste Management,industrial,Waste Management,utilities,Utility Company,wastemanagementservices,"[22951, 22952, 22958, 72804, 72828]",True,Utility Company/Waste Management
1677,Water Testing Service,professional services,Water Testing Service,utilities,Utility Company,watertestingservice,"[26221, 96187, 96193, 96195, 96203, 96205, 962...",True,Utility Company/Water Testing Service
1678,Water Treatment Equipment,industrial,Water Treatment Equipment,utilities,Utility Company,watertreatmentequipmentservices,"[106752, 112498, 120660, 1315, 33970, 6455, 64...",True,Utility Company/Water Treatment Equipment
1679,Water Treatment Plant,industrial,Water Treatment Plant,utilities,Utility Company,watertreatmentplant,"[11854, 16895, 55797, 92735, 92748, 92754, 927...",True,Utility Company/Water Treatment Plant


In [111]:
df_tree.to_csv('./data/gd-mvcr8intel-stage-gd-studio/deep-industry/df_tree_export_categories_parents.csv')

## Load Data - wam_website_feature

```
SELECT create_date, domain_name, vertical, vgroup, vtemplate 
FROM AwsDataCatalog.gmode.wam_website_feature
WHERE gd_domain = 0 
ORDER BY "create_date"
```

In [4]:
df_domains = pd.read_csv("./data/gd-mvcr8intel-stage-gd-studio/deep-industry/raw/3f5d2755-6bba-46f5-8982-440ae44fd2e4.csv")

In [5]:
df_domains.shape

(11126866, 5)

In [6]:
df_domains.head()

,create_date,domain_name,vertical,vgroup,vtemplate
0,2015-12-17 14:31:50.087,agile.club,Pizza,restaurant,RestaurantRestaurants
1,2016-02-16 17:39:01.002,cynical-optimist.com,Table Tennis,sports,NaN
2,2016-02-17 18:29:16.586,elianacommisso.com,Restaurant,restaurant,RestaurantRestaurants
3,2016-03-22 10:05:53.199,saucebosses.com,Wholesale Food Products,food and drink,FoodFoodShop
4,2016-03-22 13:11:30.965,sof3gun.com,Archery and Shooting,sports,NaN


In [15]:
ser_domain_verticals = (
    df_domains
    ['vertical']
    .value_counts()
    .index
)
ser_domain_verticals

Index(['Coming Soon', 'IT Services', 'Clothing and Apparel Store',
       'Consulting', 'Blog', 'Marketing Consultant', 'Nonprofit',
       'Real Estate', 'Real Estate Agent', 'Ecommerce',
       ...
       'Scandinavian Restaurant', 'Austrian Restaurant', 'Armenian Restaurant',
       'Eastern European Restaurant', 'Kwanzaa', 'Hungarian Restaurant',
       'Norwegian Restaurant', 'Basque Restaurant', 'Finnish Restaurant',
       'Bulgarian Restaurant'],
      dtype='object', length=1607)

In [14]:
(
    df_tree
    .loc[lambda df: df['name'].isin(ser_domain_verticals)]
    .shape
)

(1556, 9)

In [20]:
(
    df_tree
    .loc[lambda df: ~df['name'].isin(ser_domain_verticals)]
    [['full_path']]
)

,full_path
0,Active Living
60,Active Living/Outdoors/Fishing Guide
108,Active Living/Sports/Sporting Goods/Cycling
109,Active Living/Sports/Sporting Goods/Bike Store
118,Active Living/Sports/Sporting Goods/Sportswear
...,...
1611,Shopping/Home and Garden Store/Home Decor/Meta...
1634,Shopping/Jewelry Store
1641,Shopping/Jewelry Store/Watches
1649,Shopping/Painting Party


In [49]:
set_df_tree_verticals = set(df_tree['name'])
set_domain_verticals = set(ser_domain_verticals)

In [26]:
len(set_domain_verticals & set_df_tree_verticals)

1556

In [31]:
len(set_domain_verticals - set_df_tree_verticals)

51

In [29]:
df_tree[lambda df: df['name'].str.contains('3d', False)]

,tree,group,name,parent,parents_path,id,photos,visible,full_path
323,3D Printing,professional services,3D Printing,bbbusinessservices,Business,3dservices,"[03bK113, 28369, 28378, 28379, 6QGAzr5, 6yYJoZ...",True,Business/3D Printing


In [30]:
df_tree[lambda df: df['name'].str.contains('abuse', False)]

,tree,group,name,parent,parents_path,id,photos,visible,full_path
1040,Abuse and Addiction Care,health/medical,Abuse and Addiction Care,health,Health and Medical,abuseaddictiontreatment,"[109174, 112587, 112737, 114341, 114823, 12178...",True,Health and Medical/Abuse and Addiction Care


In [ ]:
set_domain_verticals - set_df_tree_verticals

{'3D Printing and Services',
 'Abuse and Addiction Treatment',
 'Accounting or Bookkeeping',
 'Addiction Treatment Center',
 'Arts and Crafts',
 'Beauty and Spa',
 'Bike',
 'Bike Shop',
 'Birth Control and Family Planning Center',
 'Book Store',
 'CSA',
 'Cheese Shop',
 'Clothing and Apparel Store',
 'Coffee or Tea Shop',
 'Community and Clubs',
 'Consignment Shop',
 'Cookie Shop',
 'Cosmetics and Beauty Supply',
 'Dairy Shop',
 'Dessert Shop',
 'Electronic Store',
 'Family and Parenting Blog',
 'Financial Service',
 'Fundraiser',
 'Gift Shop',
 'Glass and Mirror Shop',
 'Hobby Shop',
 'Jewelry Shop',
 'Leather Cleaning and Repair',
 'Live and Raw Food Restaurant ',
 'Makeup',
 'Music and DVDs',
 'Musical Instruments and Sheet Music',
 'Nine-Pin Bowling',
 'Other',
 'Perfume Shop ',
 'Photographer',
 'Pie Shop',
 'Quilt Shop',
 'Racing and Race Track',
 'Ramen and Noodle Shop',
 'Sewing Shop',
 'Sports Clothing and Apparel',
 'Sports and Active Life',
 'Stationery Shop',
 'Sweet Shop',

## Load More Data - gocentral_websitebuilder_session

In [90]:
df_domains_2 = pd.read_csv("./data/gd-mvcr8intel-stage-gd-studio/deep-industry/raw/b2685d00-6d9f-445c-888c-fa64bebe2127.csv")

In [91]:
df_domains_2.head()

,domain_name,business_category,gd_domain
0,busfeildarms.godaddysites.com,pubs,1
1,pherfitcollection.com,clothing,0
2,lyfemarietaxessmittycollection.godaddysites.com,clothing,1
3,k.godaddysites.com,skincare,1
4,ictsolutions.godaddysites.com,informationtechnologycompanies,1


In [95]:
df_domains_2['business_category'].nunique()

1701

In [96]:
ser_domain_2_verticals = (
    df_domains_2
    ['business_category']
    .value_counts()
    .index
)
ser_domain_2_verticals

Index(['blog', 'clothing', 'coming_soon', 'online_store', 'cosmetics', 'itservices', 'consultantsgeneralservices', 'marketingconsultants', 'nonprofit', 'personal_gaming',
       ...
       'apps', 'Restaurant', 'Cafe', 'familylawattorney', 'artist', 'new_years_day', 'taxreturnpreparationfiling', 'pianoinstructor', 'Real Estate', 'prosrvcs-commcleaning?sc_code=1'], dtype='object', length=1701)

In [97]:
set_df_tree_vertical_ids = set(df_tree['id'].values)
len(set_df_tree_vertical_ids)

1637

In [98]:
set_domain_2_verticals = set(ser_domain_2_verticals)

In [99]:
len(set_domain_2_verticals & set_df_tree_vertical_ids)

1634

In [100]:
set_df_tree_vertical_ids - set_domain_2_verticals

{'hypnosisandhypnotherapy', 'physiotherapist', 'psychotherapists'}

In [101]:
set_domain_2_verticals - set_df_tree_vertical_ids

{'Artisan',
 'Cafe',
 'Games/Toys',
 'PRESET_HEADING',
 'Real Estate',
 'Restaurant',
 'apps',
 'artist',
 'ayesha_curry_food',
 'ayesha_curry_food_2',
 'ayesha_curry_food_3',
 'boattouragency',
 'boxing_day',
 'conference_attendee',
 'custom',
 'dan_nonprofit',
 'dan_nonprofit_2',
 'dan_nonprofit_3',
 'dentallab',
 'elohim_music',
 'elohim_music_2',
 'elohim_music_3',
 'eventphotography',
 'familylawattorney',
 'farms',
 'fathers_day',
 'faxservice',
 'foodtruckscatering',
 'generic',
 'guns_and_ammo',
 'gymnastics',
 'hearingaidproviders',
 'hearingaidrepair',
 'korestaurantcomingsoon.godaddysites.com',
 'lightning',
 'lyn_fashion',
 'lyn_fashion_2',
 'lyn_fashion_3',
 'meatpacker',
 'na',
 'new_years_day',
 'nine_pin_bowling',
 'other',
 'paloma_florist',
 'paloma_florist_2',
 'paloma_florist_3',
 'pedro',
 'personal_fashion',
 'personal_fooddrink',
 'personal_pets',
 'personal_photography',
 'photographers',
 'pianoinstructor',
 'prosrvcs-career',
 'prosrvcs-commcleaning',
 'prosrv

In [102]:
df_domains_2_valid = (
    df_domains_2
    .loc[lambda df: df['business_category'].isin(set_df_tree_vertical_ids)]
    # .value_counts()
)
df_domains_2_valid.shape

(20592855, 3)

In [103]:
(df_domains_2_valid['business_category'].value_counts() > 1000).sum()

1223

In [82]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)

In [86]:
df_domains_2_valid['business_category'].nunique()

1634

In [87]:
df_domains_2_valid.shape

(20592855, 2)

In [104]:
df_domains_2_valid.head()

,domain_name,business_category,gd_domain
0,busfeildarms.godaddysites.com,pubs,1
1,pherfitcollection.com,clothing,0
2,lyfemarietaxessmittycollection.godaddysites.com,clothing,1
3,k.godaddysites.com,skincare,1
4,ictsolutions.godaddysites.com,informationtechnologycompanies,1


In [106]:
(
    df_domains_2_valid
    .loc[lambda df: df['gd_domain'] == 0]
    .shape
)

(6796519, 3)

In [108]:
(
    df_domains_2_valid
    .loc[lambda df: df['gd_domain'] == 0]
    ['business_category']
    .value_counts() > 1000
).sum()

843

In [107]:
(
    df_domains_2_valid
    .loc[lambda df: df['gd_domain'] == 0]
    ['business_category']
    .value_counts()
    .pipe(lambda df: pd.DataFrame(df))
)

,business_category
coming_soon,241261
clothing,172764
online_store,141909
consultantsgeneralservices,125639
blog,116731
itservices,102397
nonprofit,101250
realestate,96027
marketingconsultants,70368
cosmetics,65910
